**need some %run command and uncomment the model load before submitting**

In [1]:
import sys,json,re,string
from typing import Callable, Dict, List, Set, Tuple
from elasticsearch import Elasticsearch
import gensim.downloader as api
import gensim
import datetime
import csv
sys.path.insert(0, 'extract_features')
from extract_No1to5_features import extract_features_1to5
from extract_No11to12_features import extract_features_11to12
from extract_No13to15_features import TypeTaxonomy, extract_features_13to15
from extract_No16_feature import extract_features_16
from extract_No17to19_features import get_analyze,extract_features_17to19
from extract_No20to22_features import extract_features_20to22
from extract_No23to25_features_optimized import extract_features_23to25

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\junec\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\junec\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# Reduce logging level.
import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.WARNING)
logging.basicConfig(level=logging.ERROR)

logging.getLogger("elasticsearch").disabled = True
# es_log = logging.getLogger("elasticsearch")
# es_log.setLevel(logging.CRITICAL)
es= Elasticsearch(timeout=600)
es.info()

{'name': 'LAPTOP-ADBLIUPR',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': '5VELmBwJTk-urTuhZdTgew',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'zip',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [3]:
#%run extract_features/extract_No11to12_features

**for my computer , need the following enviroment to run this script**
<br>
/c/Users/junec/anaconda3/python
<br>
/c/Users/junec/anaconda3/Scripts/pip

In [4]:
!which python
!which pip

/c/Users/junec/anaconda3/python
/c/Users/junec/anaconda3/Scripts/pip


In [5]:
#load model
try:
    model_loaded = gensim.models.keyedvectors.KeyedVectors.load('googleNews.d2v')
except:
    model_loaded = api.load('word2vec-google-news-300')
    model_loaded.save('googleNews.d2v')
    model_loaded = gensim.models.keyedvectors.KeyedVectors.load('googleNews.d2v')

2021-11-20 06:41:46,913 : INFO : loading KeyedVectors object from googleNews.d2v
2021-11-20 06:41:48,353 : INFO : loading vectors from googleNews.d2v.vectors.npy with mmap=None
2021-11-20 06:42:08,299 : INFO : KeyedVectors lifecycle event {'fname': 'googleNews.d2v', 'datetime': '2021-11-20T06:42:08.267898', 'gensim': '4.1.2', 'python': '3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'loaded'}


In [6]:
#load files
filepath="data/DBpedia_map_type_entities.json"
with open(filepath, 'r', encoding='utf-8') as file:
    DBpedia_map_type_entities = json.load(file)
    
filepath="data/training_types.json"
with open(filepath,encoding='utf-8') as json_file:
    training_map_type_questions = json.load(json_file)

filepath="data/ElasticSearch_map_type_docID.json"
with open(filepath, 'r',encoding='utf-8') as f:
    docID_DBOtype_dict = json.load(f)

filepath="../smart-dataset/datasets/DBpedia/smarttask_dbpedia_train.json"
with open(filepath, 'r') as f:
    smarttask_dbpedia_train = json.load(f)

typeobj=TypeTaxonomy("data/dbpedia_types.tsv")

In [7]:
# type_list=list(typeobj._types.keys())
# type_list[1:]
# len(type_list[1:]),len(smarttask_dbpedia_train) ,smarttask_dbpedia_train[1]

In [8]:
# dp_type="dbo:MilitaryPerson"
# question="Does the shelf life of spinach equal 8?"

# dp_type="dbo:MusicFestival"
# question="When was Bibi Andersson music festival married to Per Ahlmark very green?"

# extract_features_1to5(DBpedia_map_type_entities,dp_type,question,es)
# features_1to5=extract_features_1to5(DBpedia_map_type_entities,dp_type,question,es)
# extract_features_11to12(dp_type, question,docID_DBOtype_dict,es)
# extract_features_13to15(typeobj,dp_type)
# print(extract_features_16(DBpedia_map_type_entities,dp_type))

# extract_features_17to19(training_map_type_questions,dp_type,question)
# extract_features_20to22(dp_type,question)
# extract_features_23to25(model_loaded,dp_type, question, mode="Euclidean")
# extract_features_23to25(model_loaded,dp_type, question, mode="similarities")

In [9]:
def preprocess(doc: str) -> str:
    """Preprocesses text to prepare it for feature extraction.

    Args:
        doc: String comprising the unprocessed contents of some email file.

    Returns:
        String comprising the corresponding preprocessed text.
    """
    re_html = re.compile("<[^>]+>")
    doc = re_html.sub(" ", doc)
    #remove pure digits 
    doc=re.sub(r"(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b","",doc)
    # Replace punctuation marks (including hyphens) with spaces.
    for c in string.punctuation:
        doc = doc.replace(c, " ")
    #return doc.lower()
    return doc

In [10]:
def extract_features_type_label(
    question:str,
    dp_type: str,
    training_map_type_questions:Dict,
    model_loaded:gensim.models.keyedvectors.KeyedVectors,
    es: Elasticsearch,
    analyze,X,terms_corpus,
    add_extra_features:bool=False
) -> List[float]:
    """Extracts features of a query and document pair.

        Args:
            query: string.
            dp_type: DBO type.
            es: Elasticsearch object instance.

        Returns:
            List of extracted feature values in a fixed order.
    """  
    features_17to19=extract_features_17to19(analyze,X,terms_corpus,training_map_type_questions,dp_type,question)
    feature_vect=list(features_17to19.values())
    
    features_20to22=extract_features_20to22(dp_type,question)
    feature_vect.extend(list(features_20to22.values()))
    
    features_23to25=extract_features_23to25(model_loaded,dp_type, question, mode="Euclidean")
    feature_vect.extend(list(features_23to25.values()))

    if add_extra_features:
        features_23to25_variant=extract_features_23to25(model_loaded,dp_type, question, mode="similarities")
        feature_vect.extend(list(features_23to25_variant.values()))
    
    return feature_vect

In [11]:
# analyze,X,terms_corpus=get_analyze(training_map_type_questions)
# DBOtype="dbo:MusicFestival"
# question="When was Bibi Andersson music festival married to Per Ahlmark very green?"
# question="Who is {famous for} of {writers} of {To the Christian Nobility of the German Nation} ?"
# extract_features_type_label(question,
#                                                         DBOtype,
#                                                         training_map_type_questions,
#                                                         model_loaded,
#                                                         es,
#                                                         analyze,X,terms_corpus) 

In [12]:
# file = open("data/baseline_features.csv")
# csvreader = csv.reader(file)
# rows = []
# for row in csvreader:
#         rows.append(row)
# rows

# type(rows[3])
# rows[3]


In [ ]:
def prepare_training_data_type_label(  smarttask_dbpedia_train:List[Dict],
                                        training_map_type_questions:Dict[str,str],
                                        model_loaded:gensim.models.keyedvectors.KeyedVectors,
                                        es: Elasticsearch,
                                        add_extra_features:bool=False)-> Tuple[List[List[float]], List[int]]:
    """
    """
#     X_train=[]
#     y_train=[]
    all_DBOtype=list(typeobj._types.keys())
    #remove the first type:"thing"
    all_DBOtype=all_DBOtype[1:]
    analyze,X,terms_corpus=get_analyze(training_map_type_questions)
    
    count=0
    j=0
#     file_X = open("data/X_train_type_label.csv", 'w', newline='')
#     writer_X = csv.writer(file_X)
#     file_y = open("data/y_train_type_label.csv", 'w', newline='')
#     writer_y = csv.writer(file_X)
    with open("data/training_type_label.csv", 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)


        for entry in smarttask_dbpedia_train:
            if j%100==0:
                print("--------------------j:",j)

            j+=1
            if entry['question']==None:
                continue

            question_processed=preprocess(entry['question'])

            if entry['category']=='resource':
                if count%30==0:
                    print("-----------------",datetime.datetime.now())
                    print("--------------------count:",count)  
#                   print(f'--------------length of X_train:{len(X_train)} y_train: {len(y_train)}')
                count+=1
                #print("question_processed:",question_processed)
                for DBOtype in entry['type']: 
                    try:
                        features=extract_features_type_label(question_processed,
                                                            DBOtype,
                                                            training_map_type_questions,
                                                            model_loaded,
                                                            es,
                                                            analyze,X,terms_corpus) 
                    except BaseException as err:
                        print("------------error for type:",DBOtype,entry['question'])
                        print(f"Unexpected {err}, {type(err)}")  
                        raise
#                     X_train.append(features)
#                     y_train.append(1)


#                     writer_X.writerow(features)
#                     writer_y.writerow(y_train)
                    writer.writerow(features)
                    writer.writerow([1])

                #deal with top 100 documents
                hits = es.search(
                    index="dbpdiea_type_centric", q=question_processed, _source=True, size=30
                )["hits"]["hits"]
                rank_list= [hit['_source']["type"] for hit in hits]
                #print(rank_list)

                for DBOtype in rank_list:
                    if DBOtype not in entry['type']:
                        try:
                            features=extract_features_type_label(question_processed,
                                                            DBOtype,
                                                            training_map_type_questions,
                                                            model_loaded,
                                                            es,
                                                            analyze,X,terms_corpus) 
                        except BaseException as err:
                            print("------------error for type:",DBOtype,entry['question'])
                            print(f"Unexpected {err}, {type(err)}")  
                            raise
                            
                        writer.writerow(features)
                        writer.writerow([0])
                        #print(features)
#                         X_train.append(features)
#                         y_train.append(0)
#                         writer_X.writerow(features)
#                         writer_y.writerow(y_train)

    #                 if count%100000==0:
    #                     print(f'--------------{count} is finished')
    #                     if len(X_train)%500==0:
    #                         print("-----------------",datetime.datetime.now())
    #                         print(f'--------------length of X_train:{len(X_train)} y_train: {len(y_train)}')

    print(f'{count} features has been saved,{j} questions have been processed')
#     file_X.close()
#     file_y.close()
    return None



prepare_training_data_type_label(smarttask_dbpedia_train,
                                        training_map_type_questions,
                                        model_loaded,
                                        es)

--------------------j: 0
----------------- 2021-11-20 06:42:12.250400
--------------------count: 0


C:\Users\junec\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\junec\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


----------------- 2021-11-20 06:45:28.287709
--------------------count: 30
--------------------j: 100
----------------- 2021-11-20 06:48:19.671073
--------------------count: 60
----------------- 2021-11-20 06:50:41.771298
--------------------count: 90
--------------------j: 200
----------------- 2021-11-20 06:52:39.138388
--------------------count: 120
----------------- 2021-11-20 06:55:40.045112
--------------------count: 150
--------------------j: 300
----------------- 2021-11-20 06:57:42.145476
--------------------count: 180
----------------- 2021-11-20 07:00:42.011795
--------------------count: 210
--------------------j: 400
----------------- 2021-11-20 07:03:02.496809
--------------------count: 240
----------------- 2021-11-20 07:05:32.707688
--------------------count: 270
--------------------j: 500
----------------- 2021-11-20 07:08:14.221916
--------------------count: 300
----------------- 2021-11-20 07:11:02.133023
--------------------count: 330
--------------------j: 600
-----

--------------------j: 5000
----------------- 2021-11-20 10:34:45.248544
--------------------count: 2730
----------------- 2021-11-20 10:36:58.418805
--------------------count: 2760
--------------------j: 5100
----------------- 2021-11-20 10:39:43.734570
--------------------count: 2790
----------------- 2021-11-20 10:42:25.007195
--------------------count: 2820
--------------------j: 5200
----------------- 2021-11-20 10:44:49.134152
--------------------count: 2850
--------------------j: 5300
----------------- 2021-11-20 10:47:14.321605
--------------------count: 2880
----------------- 2021-11-20 10:49:57.401339
--------------------count: 2910
--------------------j: 5400
----------------- 2021-11-20 10:52:11.113223
--------------------count: 2940
----------------- 2021-11-20 10:54:25.981928
--------------------count: 2970
--------------------j: 5500
----------------- 2021-11-20 10:57:11.950312
--------------------count: 3000
--------------------j: 5600
----------------- 2021-11-20 10:59

In [ ]:
%run prepare_training_data_baseline.ipynb

In [ ]:
# with open("../data/X_train_type_label.json", 'w') as f:
#   json.dump(X_train, f)
# with open("../data/X_train_type_label.json", 'w') as f:
#   json.dump(y_train, f)
# print("X_train,y_train are saved as json files")

In [ ]:
# es=Elasticsearch()
# es.get(index='dbpdiea_type_centric',id="3")

In [ ]:
# index='dbpdiea_type_centric'
# field="abstract"
# tv = es.termvectors(index=index, doc_type="_doc", id="1", fields=field, term_statistics=True) 
# collection_len=tv["term_vectors"][field]['field_statistics']['sum_ttf']  
# #total number of entity in the collections
# doc_number=tv["term_vectors"][field]['field_statistics']['doc_count'] 
# #average document length
# avgdl=collection_len/doc_number
# collection_len,doc_number,avgdl

In [ ]:
# word="theatre"
# #model_loaded[word]
# model_loaded.most_similar(positive=[word], topn=1)